In [ ]:
#Instalando a biblioteca que providência acesso aos modelos generativos de IA Google
#-q é um comando que vem do "quiet", onde a execução do bloco não irá renderizar muitas coisas, fazendo com quea interface fique mais limpa
!pip -q install google-genai

In [ ]:
#Configurando a API Key do Google AI Studio no código
#importando módulos do OS que permite acessar componentes do sistema operação, como por exemplo, variáveis do sistema
import os
#importando função userdata do modulo do google Colab - Especificamente pra poder coletar e trabalhar com dados sensíveis, exemplo a API Key que guardamos no secret
from google.colab import userdata
#atribuindo a variável do sistema o valor da API Key que guardamos no Secrets aqui no Colab
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
#Configurando o SDK do Gemini
#importando o módulo genai do pacote google
from google import genai
#criando uma instância para fazer requisiçõesà API do Google
client = genai.Client()
#Definindo qual o modelo de IA vamos usar e armazenando em uma variável para não precisarmos escrever o modelo toda vez que precisar
IA_Model = 'gemini-2.0-flash'

In [ ]:
#Importando funcionalidade HTML e Markdown do módulo IPython
#HTML e Markdown são puramente funções visuais, que ajudam a fazer com que o notebook renderize o conteúdo com um visual melhor - para HTML e para textos formatados, por exemplo.
from IPython.display import HTML, Markdown

In [ ]:
#executando o pip diferente, com % ao invés de ! - Ambos funcionam.
#Instalando o Framework ADK (Agents Development Kit) do Google
%pip -q install google-adk

In [ ]:
#Importando algumas funcionalidades do ADK para criar uma esteira de agentes.
#Importando o Agent que será responsável pela realização das ações e interações com o modelo de IA
from google.adk.agents import Agent
#Importando o Runner que será responsável pela orquestração das interações e múltiplos agentes.
from google.adk.runners import Runner
#Importando o MemoryService que será responsável por garantir o histórico e o contexto da conversa dentro de uma única execução.
from google.adk.sessions import InMemorySessionService
#Importando a função google_search que vai permitir os agentes a usar a busca do Google e porencializar os resultados
from google.adk.tools import google_search
#Importando o módulo types que será usado para representar diferentes tipos de conteúdo ou partes de conteúdo ao interagir com a API de IA.
from google.genai import types
#Importando a classe date para trabalhar com datas e horas
from datetime import date
#Importando o módulo textwrap, responsável por funções de quebra e formatação de texto simples - Melhora a identação da resposta para tornar o texto mais legível.
import textwrap
#Importando a biblioteca requests para interagir com serviços web, APIs ou serviços externos
import requests
#Importando o módulo warnings com a linha de baixo controlando como os avisos são tratados no ambiente. Ignore significa que os alertas serão suprimidos e não exibidos no output.
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Criando uma função que vai enviar uma mensagem para um agente via Runner e retornar a resposta final
#call_agent espera receber o Agent e a mensagem. str indica que a função deve retornar uma string (texto)
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    #definindo a variável final_response com um valor em branco, para concatenarmos as partes do texto nela e exibi-la futuramente.
    final_response = ""

    #Loop for iniciado para iterar os eventos retornados durante a execução do agente
    #runner.run() é a função que executa o agente, com um ID de usuário e Sessão e o conteúdo da mensagem.
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        #verificação se é a resposta final. Validação feita por true ou false. True representa que o agente concluiu o processamento e está fornecendo a resposta final.
        if event.is_final_response():
          #Se o evento for a resposta final, esse loop vai iterar sobre as diferentes partes do conteúdo da resposta - podendo ser texto, imagens, etc
          for part in event.content.parts:
            #Essa linha verifica se a parte atual (part) contém texto, garantindo que estamos lidando apenas com partes que contém alguma coisa.
            if part.text is not None:
              #se a parte tiver texto, adicionamos à variável final_response. O operador += é que faz isso. \n quebra linha.
              final_response += part.text
              final_response += "\n"
    #retorna a resposta final acumulada do agente.
    return final_response

In [ ]:
#Criando uma função chamada to_markdown, que receberá um texto, para que ele possa ser exibido de forma agradável em um ambiente Jupyter Notebook
def to_markdown(text):
  #Modifica o texto de entrada, quaisquer marcadores de lista (bullet points) representados pelo caractere '•' por ' *'
  text = text.replace('•', '  *')
  #textwrap.indent(text, '> ', predicate=lambda _: True): Esta parte usa a função indent do módulo textwrap. Ela indenta cada linha do text de entrada com o prefixo '> '. Esta é uma maneira comum de formatar citações em Markdown. O predicate=lambda _: True garante que todas as linhas sejam indentadas, não apenas algumas específicas.
  #Markdown(...): A saída da função textwrap.indent (o texto indentado) é então envolvida na função Markdown. Esta função, importada de IPython.display, instrui o ambiente Jupyter Notebook a interpretar a string contida como Markdown e renderizá-la de acordo.
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# --- Agente 1: Pesquisador de Projetos Sociais --- #
#Criando o primeiro Agent chamado pesquisador_social, que espera receber 2 parametros: Localização e Data
def pesquisador_social(localizacao, data_de_hoje):

    #Atribui à variável pesquisador, o objeto agente com os parametros nome, modelo de IA, descrição, ferramentas e instrução.
    pesquisador = Agent(
        name="pesquisador_social",
        model="gemini-2.0-flash",
        description="agente que pesquisa sobre ações sociais na região",
        tools=[google_search],
        instruction="""Você é um especialista em projetos sociais. Sua responsabilidade é identificar a região e estudar sobre
        os possíveis projetos sociais, voluntariado, ações sociais etc. que podem estar acontecendo na região.
        Você deve retornar uma lista com os projetos encontrados. Caso existam inscrições, fornecer data de abertura e data limite também.
        Preciso que você também de sugestões de possíveis projetos sociais para se criar na região, baseado no que foi encontrado e o que não foi encontrado, mas que gerariam impacto para a população.
        Caso não encontre nenhum projeto, retorne uma mensagem informando que não encontrou nenhum projeto.
        """
    )

    #Armazenando na variável entrada_para_pesquisa o que for digitado pelo usuário para localização e a data de hoje
    entrada_para_pesquisa = f"Localização: {localizacao}\n Data de hoje: {data_de_hoje}"

    #Armazena na variável projetos a resposta da IA depois de receber a localização e a data e pesquisar sobre o assunto.
    projetos_achados = call_agent(pesquisador, entrada_para_pesquisa)
    #Retorna o valor obtido na variável
    return projetos_achados

In [ ]:
# --- Agente 2: Organizador de Projetos Sociais --- #
#Criando o segundo Agent chamado organizador_social, que espera receber 2 parametros: Localização e Projetos (vindo do agente anterior)
def organizador_social(localizacao, projetos_achados):

    #Atribui à variável organizador, o objeto agente com os parametros nome, modelo de IA, descrição, ferramentas e instrução.
    organizador = Agent(
        name="organizador_social",
        model="gemini-2.0-flash",
        description="organiza e filtra a lista de projetos encontrados",
        tools=[google_search],
        instruction="""Você é um organizador de projetos sociais. Sua responsabilidade é receber os projetos encontrados, analisar e filtrá-los por área
        para que o usuário consiga localizar, de forma clara, quais projetos fazem mais sentido pra ele. Você deve retornar uma lista com os projetos encontrados, filtrados por área, com título, descrição, se possível incluir telefone para contato com os responsáveis.
        Caso exista, aplique também as datas necessárias, como data de abertura e data limite de inscrição.
        """
    )

    #Armazenando na variável entrada_para_organizacao o que for digitado pelo usuário para localização e os projetos encontrados
    entrada_para_organizacao = f"Localização: {localizacao}\n Projetos: {projetos_achados}"

    #Armazena na variável projetos_organizados a resposta da IA depois de receber a localização e os projetos do agente anterior
    projetos_organizados = call_agent(organizador, entrada_para_organizacao)
    #Retorna o valor obtido na variável
    return projetos_organizados

In [ ]:
#Importanto a biblioteca pandas e atribuindo o nome pd a ela. Ela serve para manipular e analisar dados. Vamos usá-la para gerar um csv com os projetos sociais.
import pandas as pd
from google.colab import files


# --- Agente 3: Criação de arquivos CSV --- #
# Criando o terceiro Agent chamado analista_social, que espera receber 2 parametros: Localização e Projetos Organizados (vindo do agente anterior)
def analista_social(localizacao, projetos_organizados):

    # Atribui à variável analista, o objeto agente com os parametros nome, modelo de IA, descrição, ferramentas e instrução.
    analista = Agent(
        name="analista_social",
        model="gemini-2.0-flash",
        description="Analisa e cria arquivos csv ou xlsx para usuário final",
        tools=[google_search],
        instruction="""Você é um analista social e sua função é prover um arquivo csv para o usuário final com os projetos organizados.
        Sua resposta DEVE ser formatada EXATAMENTE como o conteúdo de um arquivo CSV.
        Inclua uma linha de cabeçalho com os nomes das colunas (por exemplo: Título,Descrição,Área Social, Público, Telefone de Contato).
        Você também deve separar no arquivo projetos que estão acontecendo e projetos que podem ser criados.
        Cada linha subsequente deve representar um projeto, com os detalhes separados por vírgulas, na mesma ordem do cabeçalho.
        Não inclua nenhum texto adicional antes ou depois do conteúdo CSV.
        """
    )

    # Armazenando na variável entrada_para_organizacao o que for digitado pelo usuário para localização e os projetos encontrados
    entrada_para_criacao = f"Localização: {localizacao}\n Projetos: {projetos_organizados}"

    # Armazena na variável criacao_arquivo_csv_content a resposta da IA formatada como CSV
    criacao_arquivo_csv_content = call_agent(analista, entrada_para_criacao)

    # Imprime a string retornada para depuração (opcional)
    print("Conteúdo CSV retornado pelo agente:")
    print(criacao_arquivo_csv_content)

    # --- Definir o nome do arquivo ---
    nome_arquivo = f"projetos_sociais_{localizacao}.csv"

    # --- Escrever o conteúdo CSV em um arquivo ---
    try:
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            f.write(criacao_arquivo_csv_content)
    except Exception as e:
        return f"Ocorreu um erro ao escrever o arquivo: {e}"

    # --- Disponibilizar o arquivo para download ---
    try:
        files.download(nome_arquivo)
        return f"Arquivo '{nome_arquivo}' criado e disponível para download."
    except Exception as e:
        return f"Ocorreu um erro ao baixar o arquivo: {e}"

In [ ]:
#Obtém a data da execução e armazena na variável data_de_hoje
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Bem-Vindo ao Assistente Social! 🚀")

# --- Obter a localização do Usuário ---
localizacao = input("❓ Por favor, digite sua localização: ")

# Inserir lógica do sistema de agentes ################################################

projetos_achados = pesquisador_social(localizacao, data_de_hoje)
print('\n--- Projetos Disponíveis ---\n')
display(to_markdown(projetos_achados))
print('-------------------------------------------------')

organizacao_projetos = organizador_social(localizacao, projetos_achados)
print('\n--- Projetos Organizados ---\n')
display(to_markdown(organizacao_projetos))
print('-------------------------------------------------')

criacao_arquivo = analista_social(localizacao, organizacao_projetos)
print('\n--- Arquivo Criado ---\n')
display(to_markdown(criacao_arquivo))
print('-------------------------------------------------')